In [1]:
import pandas as pd
from pymongo import MongoClient
import pymongo
import numpy as np

In [2]:
class ConectColl:
    def __init__(self):
        self.client = MongoClient()
        self.db = self.client['Bycicle_Accidents']
        self.acc = self.db['Historical']
        self.test = self.db['Test']

coll = ConectColl()
bici = pd.read_csv("../input/clean_data/bici_clean.csv")

In [2]:
bici = pd.read_csv("../input/clean_data/bici_clean.csv")

In [3]:
bici.head()

,fecha,dia,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,lon,lat
0,2019-01-01 14:00:00,2019-01-01 00:00:00,2019,Tarde,Festivo,Colision,Moderada,Despejado,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,-3.681209,40.428990
1,2019-01-02 21:00:00,2019-01-02 00:00:00,2019,Noche,Laborable,Colision,Grave,Despejado,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,-3.654527,40.466921
2,2019-01-03 14:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,-3.623423,40.381971
3,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939
4,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939


In [4]:
len(bici.loc[bici.lesividad=='Leve'])

1759

In [5]:
bici.shape

(2198, 12)

In [9]:
lst_h = list(bici.horario.unique())
lst_f = list(bici.festividad.unique())
lst_m = list(bici.meteo.unique())
lst_l = list(bici.lesividad.unique())

for h in lst_h:
    for f in lst_f:
        for m in lst_m:
            longitud = len(bici.loc[(bici.horario==h)&(bici.festividad==f)&(bici.meteo==m)])
            print(f'números en la combinación {h} {f} {m}:{longitud}')
            for l in lst_l:
                longitud = len(bici.loc[(bici.horario==h)&(bici.festividad==f)&(bici.meteo==m)&(bici.lesividad==l)])
                print(f'\t{l}:{longitud}')
            print('')

números en la combinación Tarde Festivo Despejado:75
	Moderada:21
	Grave:12
	Leve:42
	Fallecido:0

números en la combinación Tarde Festivo Lluvia:10
	Moderada:1
	Grave:0
	Leve:9
	Fallecido:0

números en la combinación Tarde Laborable Despejado:511
	Moderada:114
	Grave:34
	Leve:362
	Fallecido:1

números en la combinación Tarde Laborable Lluvia:54
	Moderada:7
	Grave:2
	Leve:45
	Fallecido:0

números en la combinación Tarde Vispera Despejado:92
	Moderada:19
	Grave:14
	Leve:59
	Fallecido:0

números en la combinación Tarde Vispera Lluvia:8
	Moderada:1
	Grave:1
	Leve:6
	Fallecido:0

números en la combinación Noche Festivo Despejado:52
	Moderada:8
	Grave:2
	Leve:42
	Fallecido:0

números en la combinación Noche Festivo Lluvia:4
	Moderada:0
	Grave:1
	Leve:3
	Fallecido:0

números en la combinación Noche Laborable Despejado:159
	Moderada:44
	Grave:17
	Leve:98
	Fallecido:0

números en la combinación Noche Laborable Lluvia:18
	Moderada:1
	Grave:1
	Leve:16
	Fallecido:0

números en la combinación Noch

In [5]:
num_lon = 1/93.75*0.5
num_lat = 1/125*0.5

bici['num_acc']=100

for i, r in bici.loc[(bici.lesividad=='Leve')&(bici.meteo!='Lluvia')].iterrows():
    accs = len(bici.loc[
        (bici.horario==r['horario'])&(bici.festividad==r['festividad'])&(bici.meteo==r['meteo'])&
        (bici.lon>=(r['lon']-num_lon))&(bici.lon<=(r['lon']+num_lon))&
        (bici.lat>=(r['lat']-num_lat))&(bici.lat<=(r['lat']+num_lat))])
    bici.loc[i,'num_acc']=accs

In [6]:
bici['mark']= int()
bici.loc[(bici.lesividad=='Leve')&(bici.festividad=='Laborable')&(bici.meteo=='Despejado')&(bici.num_acc<=5), 'mark']=1

In [7]:
bici.mark.value_counts()

0    1627
1     571
Name: mark, dtype: int64

In [10]:
bici = bici.loc[bici.num_acc>2]
bici = bici.loc[bici.mark==0]
bici.drop(['num_acc', 'mark'], axis=1, inplace = True)

In [33]:
bici.shape

(1515, 14)

In [3]:
# bici_test = bici.loc[(bici.horario=='Tarde')&(bici.festividad=='Laborable')&(bici.meteo=='Lluvia')]

In [4]:
# bici_test.head()

In [5]:
# min_lat=int((bici_test.lat.min()-0.01)*400)
# max_lat=int((bici_test.lat.max()+0.01)*400)

# min_lon=int((bici_test.lon.min()-0.01)*300)
# max_lon=int((bici_test.lon.max()+0.01)*300)


In [6]:
# no_damage = pd.DataFrame(columns=['horario','festividad','lesividad','meteo','lon', 'lat'])

# for lat in range(min_lat, max_lat):
#     for lon in range (min_lon, max_lon):
#         no_damage = no_damage.append({
#             'horario':'Tarde',
#             'festividad':'Laborable',
#             'lesividad':'No',
#             'meteo':'Lluvia',
#             'lon':lon/300,
#             'lat':lat/400,
#         }, ignore_index=True)



In [13]:
bici.loc[(bici.horario=='Mañana')&(bici.festividad=='Vispera')&(bici.meteo=='Lluvia')]

,fecha,dia,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,lon,lat


In [12]:
lst_h = list(bici.horario.unique())
lst_f = list(bici.festividad.unique())
lst_m = list(bici.meteo.unique())

no_damage = pd.DataFrame(columns=['horario','festividad','lesividad','meteo','lon', 'lat'])

for h in lst_h:
    print(h)
    for f in lst_f:
        print(f)
        for m in lst_m:
            print(m)
            bici_test = bici.loc[(bici.horario==h)&(bici.festividad==f)&(bici.meteo==m)]
            print(bici_test.lat.min())
            min_lat=int((bici_test.lat.min()-0.0002)*125)
            max_lat=int((bici_test.lat.max()+0.0002)*125)

            min_lon=int((bici_test.lon.min()-0.00015)*93.75)
            max_lon=int((bici_test.lon.max()+0.00015)*93.75)
            
            for lat in range(min_lat, max_lat):
                for lon in range (min_lon, max_lon):
                    no_damage = no_damage.append({
                        'horario':h,
                        'festividad':f,
                        'lesividad':'No',
                        'meteo':m,
                        'lon':lon/93.75,
                        'lat':lat/125,
                    }, ignore_index=True)

Tarde
Festivo
Despejado
40.36453909999999
Lluvia
40.4609216
Laborable
Despejado
40.34008970000001
Lluvia
40.3652321
Vispera
Despejado
40.34585999999999
Lluvia
40.4162424
Noche
Festivo
Despejado
40.38782
Lluvia
40.4094574
Laborable
Despejado
40.36318170000001
Lluvia
40.3896342
Vispera
Despejado
40.3627995
Lluvia
40.3434084
Mañana
Festivo
Despejado
40.35628270000001
Lluvia
40.38812
Laborable
Despejado
40.33439370000001
Lluvia
40.382571
Vispera
Despejado
40.36734000000001
Lluvia
nan


ValueError: cannot convert float NaN to integer

In [26]:
no_damage.shape

(2403, 6)

In [13]:
no_damage.shape

(3812, 6)

In [10]:
bici.head()

,fecha,dia,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,lon,lat
0,2019-01-01 14:00:00,2019-01-01 00:00:00,2019,Tarde,Festivo,Colision,Moderada,Despejado,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,-3.681209,40.428990
1,2019-01-02 21:00:00,2019-01-02 00:00:00,2019,Noche,Laborable,Colision,Grave,Despejado,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,-3.654527,40.466921
2,2019-01-03 14:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,-3.623423,40.381971
3,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939
4,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939


In [16]:
num_lon = 1/93.75*3/4
num_lat = 1/125*3/4

no_damage['num_acc']=int()

for i, r in no_damage.iterrows():
    accs = len(bici.loc[
        (bici.horario==r['horario'])&(bici.festividad==r['festividad'])&(bici.meteo==r['meteo'])&
        (bici.lon>=(r['lon']-num_lon))&(bici.lon<=(r['lon']+num_lon))&
        (bici.lat>=(r['lat']-num_lat))&(bici.lat<=(r['lat']+num_lat))])
    no_damage.loc[i,'num_acc']=accs
    

In [18]:
no_damage.num_acc.value_counts().head()
#7125/93.75 *3/4

0    2399
1     645
2     284
3     159
4      85
Name: num_acc, dtype: int64

In [15]:
no_damage.num_acc.value_counts().head()
#125/93.75

0    1923
1     690
2     426
3     185
4     132
Name: num_acc, dtype: int64

In [11]:
no_damage.num_acc.value_counts().head()
#150/112.5

0    3172
1    1016
2     477
3     228
4     142
Name: num_acc, dtype: int64

In [22]:
no_damage = no_damage.loc[no_damage.num_acc==0]

In [23]:
no_damage.shape

(2399, 7)

In [8]:
def geopoint(lon, lat):
    return {"type": "Point", "coordinates": [lon, lat]}


In [9]:
no_damage['Localizacion'] = np.vectorize(geopoint)(no_damage['lon'], no_damage['lat'])

coll.test.insert_many(no_damage.to_dict('record'))
coll.test.create_index([('Localizacion', pymongo.GEOSPHERE)])

'Localizacion_2dsphere'

In [ ]:
def num_horario(value):
    dict_hora = {
        'Mañana': 1,
        'Tarde': 2,
        'Noche': 3
    }
    return dict_hora[value]

def num_fest(value):
    dict_fest = {
        'Laborable': 1,
        'Vispera': 2,
        'Festivo': 3
    }
    return dict_fest[value]

def prepare_df(data):
    data.drop(['fecha', 'dia', 'year', 'tipo_accidente',
               'distrito', 'direccion'], axis=1, inplace=True)
    data.horario = data.horario.apply(lambda x: num_horario(x))
    data.festividad = data.festividad.apply(lambda x: num_fest(x))
    data = pd.get_dummies(data, columns=['meteo'], prefix_sep='', prefix='')
    return data

In [5]:
data = prepare_df(bici)

In [5]:
bici.head()

,fecha,dia,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,lon,lat
0,2019-01-01 14:00:00,2019-01-01 00:00:00,2019,Tarde,Festivo,Colision,Moderada,Despejado,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,-3.681209,40.428990
1,2019-01-02 21:00:00,2019-01-02 00:00:00,2019,Noche,Laborable,Colision,Grave,Despejado,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,-3.654527,40.466921
2,2019-01-03 14:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,-3.623423,40.381971
3,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939
4,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939


In [5]:
all_acc = coll.acc.find()
df = pd.DataFrame(all_acc)

In [6]:
df.head()

,_id,fecha,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,localizacion
0,5e11db4d64114c18c0e3a5c2,2019-01-01 14:00:00,2019,Tarde,Festivo,Colision,Moderada,Despejado,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,"{'type': 'Point', 'coordinates': [-3.681209, 4..."
1,5e11db4d64114c18c0e3a5c3,2019-01-02 21:00:00,2019,Noche,Laborable,Colision,Grave,Despejado,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,"{'type': 'Point', 'coordinates': [-3.654527, 4..."
2,5e11db4d64114c18c0e3a5c4,2019-01-03 14:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,"{'type': 'Point', 'coordinates': [-3.623423, 4..."
3,5e11db4d64114c18c0e3a5c5,2019-01-03 13:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,"{'type': 'Point', 'coordinates': [-3.608276, 4..."
4,5e11db4d64114c18c0e3a5c6,2019-01-03 13:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,"{'type': 'Point', 'coordinates': [-3.608276, 4..."


In [7]:
lluvia_tarde = coll.acc.find({'$and':[{"horario": "Tarde"}, {'meteo':'Lluvia'}]})
df_llt = pd.DataFrame(lluvia_tarde)

In [8]:
df_llt.head()

,_id,fecha,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,localizacion
0,5e11db4d64114c18c0e3a5ee,2019-01-29 16:00:00,2019,Tarde,Laborable,Caida,Moderada,Lluvia,CENTRO,PASEO. RECOLETOS 23 Madrid,"{'type': 'Point', 'coordinates': [-3.692396, 4..."
1,5e11db4e64114c18c0e3a678,2019-04-02 14:00:00,2019,Tarde,Laborable,Colision,Leve,Lluvia,FUENCARRAL-EL PARDO,"AUTOV. M-607, +01500S Madrid","{'type': 'Point', 'coordinates': [-3.691782, 4..."
2,5e11db4e64114c18c0e3a679,2019-04-02 14:00:00,2019,Tarde,Laborable,Colision,Leve,Lluvia,FUENCARRAL-EL PARDO,"AUTOV. M-607, +01500S Madrid","{'type': 'Point', 'coordinates': [-3.691782, 4..."
3,5e11db4e64114c18c0e3a67f,2019-04-06 20:00:00,2019,Tarde,Vispera,Caida,Moderada,Lluvia,CENTRO,CALLE VICTORIA 4 Madrid,"{'type': 'Point', 'coordinates': [-3.701736, 4..."
4,5e11db4e64114c18c0e3a686,2019-04-09 13:00:00,2019,Tarde,Laborable,Otro,Leve,Lluvia,CENTRO,PASEO. RECOLETOS / CALLE GENOVA Madrid,"{'type': 'Point', 'coordinates': [-3.690993, 4..."


In [ ]:
.find({'$and': [query, {"location": {"$near": {"$geometry": lst_loc, "$maxDistance": radio}}}]}))